In [6]:
let ResolutionFolder = __SOURCE_DIRECTORY__

[<RequireQualifiedAccess>]
type Log =
| ChangeDir of string
| GoUp
| File of int*string
| Dir of string

type ADir = {
    name: string
    dirs: ADir seq
    files: AFile seq
}
and AFile = {
    name: string
    size: int
}

type FSystem = Map<string, ADir>

let folder (fileSystem:FSystem) (log:Log) =
    fileSystem

let parseLog (line:string) =
    match line.Split(" ") with
    | [| "$"; "cd"; ".." |] -> Log.GoUp |> Some
    | [| "$"; "cd"; dir |] -> Log.ChangeDir dir |> Some
    | [| "dir"; dir |] -> Log.Dir dir |> Some
    | [| size; file |] when (Int64.TryParse(size)) |> fst -> Log.File (size |> int ,file) |> Some
    | _ -> None

File.ReadLines(ResolutionFolder + "/testcase7.txt")
|> Seq.map parseLog
|> Seq.choose id
|> Seq.fold folder Map.empty

index,type,Item,Item1,Item2
0,FSI_0014+Log+ChangeDir,/,,
1,FSI_0014+Log+Dir,a,,
2,FSI_0014+Log+File,,14848514,b.txt
3,FSI_0014+Log+File,,8504156,c.dat
4,FSI_0014+Log+Dir,d,,
5,FSI_0014+Log+ChangeDir,a,,
6,FSI_0014+Log+Dir,e,,
7,FSI_0014+Log+File,,29116,f
8,FSI_0014+Log+File,,2557,g
9,FSI_0014+Log+File,,62596,h.lst
